In [16]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


warnings.filterwarnings('ignore')


基本変数定義

In [2]:
sampling_flg=0 #サンプリング有無をコントロール
submit_flg=1 #保存するかをコントロール（サンプリングしない時のみ）

SEED=12345
sample_num=10000
fold_num=5

#train関連
train_dir='../../01_input/train.csv'
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_dir='../../01_input/test.csv'
test_drop_col=['ID_code']

#結果ファイル関連　nameは自分の名前に変更する
train_preds_dir='../../03_predict_train/oka_249-2_LightGBM_train.csv'
test_preds_dir='../../04_predict_test/oka_249-2_LightGBM_submission.csv'
save_col_name='oof_xgb'

sample_submission_dir='../../01_input/sample_submission.csv'
submission_target_col_name='target'
submission_id_col_name='ID_code'

前処理

In [3]:
#ロード
train_df=pd.read_csv(train_dir)
test_df=pd.read_csv(test_dir)

In [4]:
#サンプリング
if sampling_flg ==1:
    train_df=train_df.sample(n=sample_num,random_state=SEED)
    test_df=test_df.sample(n=sample_num,random_state=SEED)

In [5]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_train_base=train_df.drop(train_drop_col,axis=1)
y_train_base=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

In [6]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(ratio={0:len(train_df[train_df['target']==0]), 1:round(len(train_df[train_df['target']==1])*1.2)}, random_state=0)
x_train, y_train = smote.fit_sample(x_train, y_train)
len(x_train)

204020

In [12]:
x_train=pd.DataFrame(x_train)
y_train=pd.DataFrame(y_train)

モデル実行

In [13]:
#model_param
param = {
    "objective" : "binary", 
    "boost":"gbdt",
    "metric":"auc",
    "boost_from_average":"false",
    "num_threads":28,
    "learning_rate" : 0.01,
    "num_leaves" : 13,
    "max_depth":-1,
    "tree_learner" : "serial",
    "feature_fraction" : 0.05,
    "bagging_freq" : 5,
    "bagging_fraction" : 0.4,
    "min_data_in_leaf" : 80,
    "min_sum_hessian_in_leaf" : 10.0,
    "verbosity" : 1,
    'seed': 44000,
    }

In [14]:
folds = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=SEED)
oof_preds = np.zeros((len(x_train), 1))
test_preds = np.zeros((len(x_test), 1))
train_base_preds = np.zeros((len(x_train_base), 1))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(x_train.values, y_train.values)):
    print("\n")
    print("Fold {}".format(fold_))
    trn_x,trn_y = x_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x,val_y = x_train.iloc[val_idx], y_train.iloc[val_idx]
    trn_data = lgb.Dataset(trn_x,trn_y)
    val_data = lgb.Dataset(val_x,val_y)
    
    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    val_pred = clf.predict(val_x, num_iteration=clf.best_iteration)
    test_pred = clf.predict(x_test, num_iteration=clf.best_iteration)
    train_base_pred = clf.predict(x_train_base, num_iteration=clf.best_iteration)
    
    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    print(confusion_matrix(val_y, pd.DataFrame(np.round(val_pred))))

    print("Base AUC = {}".format(roc_auc_score(y_train_base, train_base_pred)))
    print(confusion_matrix(y_train_base, pd.DataFrame(np.round(train_base_pred))))
    #print("val = {}".format(val_pred)
    oof_preds[val_idx, :] = val_pred.reshape((-1, 1))
    test_preds += test_pred.reshape((-1, 1))
    train_base_preds += train_base_pred.reshape((-1, 1))
    
    print(confusion_matrix(val_y, pd.DataFrame(np.round(val_pred))))
    
test_preds /= fold_num
train_base_preds /= fold_num
roc_score = roc_auc_score(y_train, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))
print(confusion_matrix(y_train, pd.DataFrame(np.round(oof_preds))))
print("Overall Base AUC = {}".format(roc_auc_score(y_train_base, train_base_preds)))
print(confusion_matrix(y_train_base, pd.DataFrame(np.round(train_base_preds))))



Fold 0
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.884564	valid_1's auc: 0.864818
[2000]	training's auc: 0.899992	valid_1's auc: 0.876491
[3000]	training's auc: 0.910695	valid_1's auc: 0.88332
[4000]	training's auc: 0.918145	valid_1's auc: 0.887393
[5000]	training's auc: 0.923818	valid_1's auc: 0.88956
[6000]	training's auc: 0.928699	valid_1's auc: 0.890921
[7000]	training's auc: 0.933173	valid_1's auc: 0.891699
[8000]	training's auc: 0.937368	valid_1's auc: 0.892442
[9000]	training's auc: 0.941246	valid_1's auc: 0.892906
[10000]	training's auc: 0.944948	valid_1's auc: 0.893345
[11000]	training's auc: 0.948572	valid_1's auc: 0.893873
[12000]	training's auc: 0.952085	valid_1's auc: 0.894178
[13000]	training's auc: 0.955346	valid_1's auc: 0.894374
[14000]	training's auc: 0.958486	valid_1's auc: 0.8946
[15000]	training's auc: 0.961533	valid_1's auc: 0.894839
[16000]	training's auc: 0.964366	valid_1's auc: 0.894878
[17000]	training's auc: 0.96

NameError: name 'confusion_matrix' is not defined

In [17]:
    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    print(confusion_matrix(val_y, pd.DataFrame(np.round(val_pred))))

    print("Base AUC = {}".format(roc_auc_score(y_train_base, train_base_pred)))
    print(confusion_matrix(y_train_base, pd.DataFrame(np.round(train_base_pred))))
    #print("val = {}".format(val_pred)
    oof_preds[val_idx, :] = val_pred.reshape((-1, 1))
    test_preds += test_pred.reshape((-1, 1))
    train_base_preds += train_base_pred.reshape((-1, 1))
    
    print(confusion_matrix(val_y, pd.DataFrame(np.round(val_pred))))


AUC = 0.9003091702212651
[[35363   618]
 [ 2767  2057]]
Base AUC = 0.983702251160617
[[179284    618]
 [  2481  17617]]
[[35363   618]
 [ 2767  2057]]


In [18]:
test_preds

array([[0.15237118],
       [0.17867929],
       [0.15922308],
       ...,
       [0.00023065],
       [0.05452472],
       [0.10223144]])

In [19]:
#結果保存
if (submit_flg ==1 and sampling_flg==0):
    sample = pd.read_csv(sample_submission_dir)
    sample.target = test_preds[:,0].astype(float)
    sample.ID_code = test_df['ID_code']
    sample.to_csv(test_preds_dir, index=False)
    